In [1]:
import pdb
import os
import sys
from ctypes import (
    CDLL,
    POINTER,
    Structure,
    byref,
    string_at,
    c_char_p,
    c_int32,
    c_int64,
    c_uint64,
    c_ubyte,
    cast
)
import json

from ctypes.util import find_library
from typing import Optional, Union

LIB: CDLL = None

class FfiByteBuffer(Structure):
    """A byte buffer allocated by python."""
    _fields_ = [
        ("length", c_int64),
        ("data", POINTER(c_ubyte)),
    ]


class FfiError(Structure):
    """An error allocated by python."""
    _fields_ = [
        ("code", c_int32),
        ("message", c_char_p),
    ]


def _decode_bytes(arg: Optional[Union[str, bytes, FfiByteBuffer]]) -> bytes:
    if isinstance(arg, FfiByteBuffer):
        return string_at(arg.data, arg.length)
    if isinstance(arg, memoryview):
        return string_at(arg.obj, arg.nbytes)
    if isinstance(arg, bytearray):
        return arg
    if arg is not None:
        if isinstance(arg, str):
            return arg.encode("utf-8")
    return bytearray()


def _encode_bytes(arg: Optional[Union[str, bytes, FfiByteBuffer]]) -> FfiByteBuffer:
    if isinstance(arg, FfiByteBuffer):
        return arg
    buf = FfiByteBuffer()
    if isinstance(arg, memoryview):
        buf.length = arg.nbytes
        if arg.contiguous and not arg.readonly:
            buf.data = (c_ubyte * buf.length).from_buffer(arg.obj)
        else:
            buf.data = (c_ubyte * buf.length).from_buffer_copy(arg.obj)
    elif isinstance(arg, bytearray):
        buf.length = len(arg)
        if buf.length > 0:
            buf.data = (c_ubyte * buf.length).from_buffer(arg)
    elif arg is not None:
        if isinstance(arg, str):
            arg = arg.encode("utf-8")
        buf.length = len(arg)
        if buf.length > 0:
            buf.data = (c_ubyte * buf.length).from_buffer_copy(arg)
    return buf


def _load_library(lib_name: str) -> CDLL:
    lib_prefix_mapping = {"win32": ""}
    lib_suffix_mapping = {"darwin": ".dylib", "win32": ".dll"}
    try:
        os_name = sys.platform
        lib_prefix = lib_prefix_mapping.get(os_name, "lib")
        lib_suffix = lib_suffix_mapping.get(os_name, ".so")
        lib_path = os.path.join(
            os.path.dirname(os.getcwd()), f"{lib_prefix}{lib_name}{lib_suffix}"
        )
        print(f"Loading library from {lib_path}")
        return CDLL(lib_path)
    except KeyError:
        print ("Unknown platform for shared library")
    except OSError:
        print ("Library not loaded from python package")

    lib_path = find_library(lib_name)
    if not lib_path:
        if sys.platform == "darwin":
            ld = os.getenv("DYLD_LIBRARY_PATH")
            lib_path = os.path.join(ld, "liboberon.dylib")
            if os.path.exists(lib_path):
                return CDLL(lib_path)

            ld = os.getenv("DYLD_FALLBACK_LIBRARY_PATH")
            lib_path = os.path.join(ld, "liboberon.dylib")
            if os.path.exists(lib_path):
                return CDLL(lib_path)
        elif sys.platform != "win32":
            ld = os.getenv("LD_LIBRARY_PATH")
            lib_path = os.path.join(ld, "liboberon.so")
            if os.path.exists(lib_path):
                return CDLL(lib_path)

        raise Exception(f"Error loading library: {lib_name}")
    try:
        return CDLL(lib_path)
    except OSError as e:
        raise Exception(f"Error loading library: {lib_name}")


def _get_library() -> CDLL:
    global LIB
    if LIB is None:
        LIB = _load_library("agora_allosaurus_rs")

    return LIB

def _get_func(fn_name: str):
    return getattr(_get_library(), fn_name)

def _free_buffer(buffer: FfiByteBuffer):
    lib_fn = _get_func("allosaurus_byte_buffer_free")
    lib_fn(byref(buffer))


def _free_string(err: FfiError):
    lib_fn = _get_func("allosaurus_string_free")
    lib_fn(byref(err))


def _free_handle(handle: c_int64, err: FfiError):
    lib_fn = _get_func("allosaurus_create_proof_free")
    lib_fn(handle, byref(err))

In [2]:
def new_server() -> c_int64:
    err = FfiError()
    lib_fn = _get_func("allosaurus_new_server")
    lib_fn.restype = c_uint64

    handle = lib_fn(byref(err))
    if handle == 0:
        message = string_at(err.message)
        raise Exception(message)
    handle = c_uint64(handle)
    return handle
server = new_server()
server

Loading library from /home/victorh/projects/anoncreds-revocation-manager-py/libagora_allosaurus_rs.so


c_ulong(4707252880464150530)

In [3]:
def new_user(server) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_new_user")
    lib_fn(server, byref(buffer), byref(err))

    if err.code != 0:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
user = new_user(server)
user

b')\xcb\x92\x0c#m\x0fd\xda6\x1b\xa9%\xe3\xe8\xfd\x16E0i\xdc\r\xd0\xec\xe1\xb0\x83*I \xef\x81\x00\xb5>\xc5Y+\xc2\xcd\x99\xc5[\x82:F\xc6\x15\xf9\xc3\xe94\x1d\xd8\xa3\x02\x84C\xf4/,G\x05\x0c+\x02\xfam\xeev\xda;\xa3\x82\xd6\xfbs\xc9\xf1U\x96\xb8m\x0eiF\xea\xe0\xaa\xfb*9\xcf\x9a\xff\x98\x8b+\xbc[\xad\x89\xa4\x15S\xa0\x92\xc0\xe6\x91zo\xab\x9c\x10\xa8\xbd\xc3/r\xa3\xfb\x93{\xebV\x7f\xee6\x15O\x0c\xee\x88\x89\x8bu/@\xde\x1c\xe9\xd1\xeb\x19\xd5n\xa1&0\xa6K\x84\xab\xf0\xc3\xed\xd6\xf6Jv\x8f\x16\xaa\x98\x94\xf8\xc5\xd2g\x9a\xdb\xa9\xdd\x15\x7f\xd2\x8b\x8d2\xebGw\xf34$r\x9f\xed\xc6!\xe8\xd3CVR\x0f+\x08\x8e\xd4@\x9f{\x88%\x16\t]\xf9\xf5!\xbb\xcaak6t\xcbY\x8b_4U\x98\x12\xb3\xa6\xc6\xd1N\x9e\xf5N\xf3T\x98\xe8\xd1\x99\x8b\x9b\x1d^\xdc\\7\x9e\xbd\x80\xea\xac\xac\xc8\x87\xca\xfd\xfe\x837\xf2\x82\xba1H\xae.\x8a;O\xf1\x06]\x01'

In [4]:
def server_add(server, user) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_add")
    lib_fn(server, _encode_bytes(user), byref(buffer), byref(err))
    if err.code != 0:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
server_add(server, user)

b'\x98\x99\n{\xca\x16\xf93\x1b0r\xa8\x86\x06\xba#\xea}\xebx@\x81y\xd4Y\x06\x9aM\xaf\xe0\xfc\xcc\x81~\xb9\xb2\xb6\xa6\xaeh!G\xc1\t\xb2\x08\x92\xdb'

In [5]:
def server_delete(server, user) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_delete")
    lib_fn(server, _encode_bytes(user), byref(buffer), byref(err))
    if err.code != 0:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
# server_delete(server, user)

In [6]:
def user_create_witness(server, user):
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_user_create_witness")
    lib_fn(server, _encode_bytes(user), byref(buffer), byref(err))
    if err.code != 0:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
user = user_create_witness(server, user)

In [7]:
def check_witness(user):
    lib_fn = _get_func("allosaurus_user_check_witness")
    err = lib_fn(_encode_bytes(user))
    if err != 0:
        raise Exception("Witness is invalid")
    return "Witness is valid"
check_witness(user)

'Witness is valid'

In [8]:
def user_make_membership_proof(server, user) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    challenge = bytearray(os.urandom(32))
    lib_fn = _get_func("allosaurus_user_make_membership_proof")
    lib_fn(server, _encode_bytes(user), _encode_bytes(challenge), byref(buffer), byref(err))
    if err.code != 0:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
proof = user_make_membership_proof(server, user)
proof

b'\x80b\x00\x0bT\x0cYl\x92#l\x97\x13\x7f\x1c\x91d\x10\x11\xfaaYs1\xa0\xbb\x8fT/\xf5V\xed\x95A,\xe2\xec\xe8&\x83\xa3\x86_\xdb"\xde\x1f\x8f\xaeB\xda\xa5\xde\xbd\xf6\x02\xb4*\xf1\xdf\x1e6\x88%\xf7\x1e"L7\xd1\xe2\x17)\xcfn\x95\xe8p\x1f#\xae\xbbm\xf0D\n\xee>\x1b\x8fU7C*\xffo\x96\x06\xcd\\\xf4\x8eD\x91\xd965\xe7W\x9b\xc0g\x8dC\xf4p\x95\x9b\xbf\x1d\x1b\x17g\xf9}\xf5\xca\x08\xae\xec\xa2\xf1\x0f\xf3.@\x02>\xac\xe9\xe7\xc1\x01\xb8\x13aI<3\x18\x02Qq\xf0\xa8+\xa4(2\xb4\xd3\x9b0\xdd\xef\xd9K\xa2\x15\xbfN\xc2n|1\xd7\x03/-\xe7\xca(+\xd9\x99\x1dV\t#wq\xea[\xfar:\x06H\x7f\x85\xf94\xa5\x99\xfc\x1a,\xaf>\xa6L\x97\xa7p\xf3\xe8\xea,}\xd2\x05G\xf3S\x9c\x9a\xcc\x0f\xcch\xcf\xda\xff\x17\x90cf\xec\xc1\x0fQ\x8cW\xf3$\xb5\xeeh\xf5#23\xd6{\x88\'":\x9be19\xf7`b\'\xb2NT\xb5\x91Dq7\xa5\x89\xd1@ySw\x08\x03Yim?\x8e^\xc7hkl\xf9\x97\x862\xe1Q\xc7\xa0\xaf\xcd\xbd^\xcf\xbf\x97f\xe8\x99\x119\xdfrz\x00Df\x05\x18\xd8/\x96R\x8d\xde\x04\xa3\rR\xd2\x87\xa4\xc1\xbbXW\x1f\x01\xcc\xaaq\xf4gi\xe7\x08\xea\xbba\xc0\x0b\x02\xd3\xedD\x

In [9]:
def witness_check_membership_proof(server, proof) -> c_int64:
    err = FfiError()
    lib_fn = _get_func("allosaurus_witness_check_membership_proof")
    lib_fn(server, _encode_bytes(proof), byref(err))
    if err.code != 0:
        message = string_at(err.message)
        raise Exception(message)
    return "Membership proof verified successfully"
witness_check_membership_proof(server, proof)

'Membership proof verified successfully'

In [10]:
def user_update(servers, user, threshold) -> c_int64:
    array_type = c_uint64 * len(servers)
    servers = array_type(*servers)

    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_user_update")
    lib_fn(servers, len(servers), _encode_bytes(user), threshold, byref(buffer), byref(err))
    if err.code != 0:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
    

In [11]:
user_update([server], user, 1)

server handles: [4707161994661199874]
Err


Exception: b'unable to update user'

In [12]:
def server_batch_delete(server, user_list):
    user_buffer = (FfiByteBuffer * len(user_list))()
    for i, tmp_user in enumerate(user_list):
        array_type = c_ubyte * len(tmp_user)
        c_array = array_type(*tmp_user)
        user_buffer[i].length = len(tmp_user)
        user_buffer[i].data = cast(c_array, POINTER(c_ubyte))

    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_update")
    lib_fn(user_buffer, len(user_list), server, byref(buffer), byref(err))
    if err.code != 0:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer


In [13]:
user1 = new_user(server)
user2 = new_user(server)
user3 = new_user(server)  
server_add(server, user1)


b'\xb7\x11\xe8x\xb9I\xc1\xea;\xa8d_l\xbb\xb3\xf7\x95N\x04\xc5\xf3\xb31o\xecs\x04:\xe5x\x1a\xf2\xcd\xa0\xbc!I\xa1\x8e\x96\xc9Ud\xfa\xcd6Od'

In [14]:
server_batch_delete(server, [user1, user2, user3])

ds: [], vs: []


b'\x00\x00'

In [15]:
def server_get_epoch(server) -> int:
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_get_epoch")
    return lib_fn(server, byref(err))
server_get_epoch(server)

1

In [16]:
def server_get_accumulator(server) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_get_accumulator")
    lib_fn(server, byref(buffer), byref(err))
    if err.code != 0:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
server_get_accumulator(server)

b'\xa2}\x05\xcb]\x1dJ\x8e`\xa0u\x19\x89\x1c\xae\x11\x9d\xd2\xfa\xef\xc4\xe5\xe04^e\x92\xda\xebuD\xcd\xb0#\x8b\xb3\xa8\xf0x\xfb\x81\xab\x16\x80\x11\x8a\x15\x95'

In [17]:
def server_get_witness_public_key(server) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_get_witness_public_key")
    lib_fn(server, byref(buffer), byref(err))
    if err.code != 0:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
server_get_witness_public_key(server)

b'\xa2\xd4c\xe0\xceog\x9e\xe9\x03\x1b\xa6\x7f\xc0\xe4\xd4\x1e=\xe9\x1f]\xdc\xf3j\xd5`#\\\xd9\x15\xac8t\xf8\x02w~l\xbf-6\xfc\xbe_\xb6\xbc\xfe\x16\rK\xc98e3\x91\xe9P\xa1\x8cPO\xdd\xa9\x83\x12\xdb\x88\xa4\xea5\xb6 =\xb2\x1d\xb7\x9bg\x99\xf8\x91\x10\xcb\xcfT\x06!\x9b\xdc\x0f\xdfwAO%\xed'

In [18]:
def server_get_sign_public_key(server) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_get_sign_public_key")
    lib_fn(server, byref(buffer), byref(err))
    if err.code != 0:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
server_get_sign_public_key(server)

b"\x87a^R\x06L\xb4\xf9N\xe5WQ\x8e\x86\x98\xbe\xb6'\x95\xacQ\xack\x9d\xe0\xbf\x03\xe8\t\xe5.\xc9|6\x7f\xd68|\x8d\x85\x8e\xb8\x1b\x87\xa2_\xc9!\x19\xdc\xbb\x06\x90E4n\x18\xdc\xf5\x9a\x1drSA\x7f\xb6\xb4r2\x98k\xe4\xb7\xf4O\xfdh\xa9W\x81\x14-6\xa6PM\xe2\xa2E\xbcf\x0e&\x81ZA"

In [19]:
def server_get_public_keys(server) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_get_public_keys")
    lib_fn(server, byref(buffer), byref(err))
    if err.code != 0:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
server_get_public_keys(server)

b"\xa2\xd4c\xe0\xceog\x9e\xe9\x03\x1b\xa6\x7f\xc0\xe4\xd4\x1e=\xe9\x1f]\xdc\xf3j\xd5`#\\\xd9\x15\xac8t\xf8\x02w~l\xbf-6\xfc\xbe_\xb6\xbc\xfe\x16\rK\xc98e3\x91\xe9P\xa1\x8cPO\xdd\xa9\x83\x12\xdb\x88\xa4\xea5\xb6 =\xb2\x1d\xb7\x9bg\x99\xf8\x91\x10\xcb\xcfT\x06!\x9b\xdc\x0f\xdfwAO%\xed\x87a^R\x06L\xb4\xf9N\xe5WQ\x8e\x86\x98\xbe\xb6'\x95\xacQ\xack\x9d\xe0\xbf\x03\xe8\t\xe5.\xc9|6\x7f\xd68|\x8d\x85\x8e\xb8\x1b\x87\xa2_\xc9!\x19\xdc\xbb\x06\x90E4n\x18\xdc\xf5\x9a\x1drSA\x7f\xb6\xb4r2\x98k\xe4\xb7\xf4O\xfdh\xa9W\x81\x14-6\xa6PM\xe2\xa2E\xbcf\x0e&\x81ZA"